# Scream Detection

## Dataset from Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Dependencies

In [ ]:
!pip install pydub

## Feature Extraction

In [ ]:
import numpy as np

In [ ]:
from pathlib import Path

DATASET_PATH = Path('./data').resolve()
NON_SCREAM_PATH = (DATASET_PATH / 'negative').resolve()
SCREAM_PATH = (DATASET_PATH / 'positive').resolve()
ATMOSPHERIC_PATH = (DATASET_PATH / 'atmospheric_data').resolve()

In [ ]:
import librosa

def extract_features(file: Path):
    audio, sr = librosa.load(file, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    features = np.mean(mfccs.T, axis=0)

    return features

In [ ]:
classes = {
    'label': ['scream', 'non_scream', 'atmospheric'],
    'dataset_dir': [SCREAM_PATH, NON_SCREAM_PATH, ATMOSPHERIC_PATH]
}

In [ ]:
extracted_features = []

for label, dataset_dir in zip(classes['label'], classes['dataset_dir']):
    for audio_file_path in dataset_dir.iterdir():
        features = extract_features(audio_file_path)
        extracted_features.append([features, label])

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
extracted_features_audio = [features for [features, _] in extracted_features]
extracted_features_audio = np.asarray(extracted_features_audio) 

extracted_features_labels = [label for [_, label] in extracted_features]
extracted_features_labels = np.asarray(extracted_features_labels)

## Model Preprocessing & training

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# One Vs Rest Strategy
lb = LabelBinarizer()
labels = lb.fit_transform(extracted_features_labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    extracted_features_audio, labels, test_size=0.3)

In [ ]:
num_label_sets = Y_train.shape[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation

model=Sequential() 

model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_label_sets))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='scream_detection.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, Y_train,
          batch_size=32, 
          epochs=100,
          validation_data=(X_test, Y_test),
          callbacks=[checkpointer],
          verbose=1,
          shuffle = True
)

Epoch 1/100
1/5 [=====>........................] - ETA: 3s - loss: 66.1898 - accuracy: 0.2812
Epoch 1: val_loss improved from inf to 17.20799, saving model to scream_detection.hdf5
5/5 [==============================] - 1s 75ms/step - loss: 44.4253 - accuracy: 0.3333 - val_loss: 17.2080 - val_accuracy: 0.3125
Epoch 2/100
1/5 [=====>........................] - ETA: 0s - loss: 30.2741 - accuracy: 0.4062
Epoch 2: val_loss did not improve from 17.20799
5/5 [==============================] - 0s 12ms/step - loss: 28.6196 - accuracy: 0.3878 - val_loss: 19.5725 - val_accuracy: 0.3125
Epoch 3/100
1/5 [=====>........................] - ETA: 0s - loss: 33.0778 - accuracy: 0.5000
Epoch 3: val_loss improved from 17.20799 to 13.87145, saving model to scream_detection.hdf5
5/5 [==============================] - 0s 17ms/step - loss: 24.2237 - accuracy: 0.4558 - val_loss: 13.8715 - val_accuracy: 0.3281
Epoch 4/100
1/5 [=====>........................] - ETA: 0s - loss: 23.3957 - accuracy: 0.3750
Epoch 4

In [ ]:
test_accuracy = model.evaluate(X_test,Y_test,verbose=0)
test_accuracy

[0.052477069199085236, 0.984375]

## Predict

In [ ]:
def predict(file: Path, model):
    # read extract_features from feature extraction
    features = extract_features(file)
    features = features.reshape(1, -1)
    labels = ['atmospheric', 'non_scream', 'scream']
    predict_x = model.predict(features)
    classes_x = np.argmax(predict_x, axis=1)
    return labels[classes_x[0]]

In [1]:
from tensorflow import keras

model = keras.models.load_model('/content/scream_detection.hdf5')
model

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
predict(next(SCREAM_PATH.iterdir()), model)

'scream'

In [ ]:
predict(next(NON_SCREAM_PATH.iterdir()), model)

'non_scream'

In [ ]:
predict(next(ATMOSPHERIC_PATH.iterdir()), model)

'atmospheric'